In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd /content
!rm -rf cnn-biomass-regression
!git clone https://github.com/prxshxntray/cnn-biomass-regression.git
%cd cnn-biomass-regression


/content
Cloning into 'cnn-biomass-regression'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 21 (delta 7), reused 6 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (21/21), 20.70 KiB | 20.70 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/cnn-biomass-regression


In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("Running on CPU")
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torchvision.transforms as transforms
import pathlib
import os, sys
from PIL import Image
import torchvision.models as models
import torch.optim as optim
from tqdm import tqdm

from cnn_workflow_utils import (
    create_kfold,
    ImageDataset,
    TARGET_COLUMNS,
    build_model,
    calculate_global_weighted_r2,
    get_device,
    train_regression,
)


CUDA available: True
GPU: Tesla T4


Using Collab's GPU: Tesla T4

**LOADING DATA**

In [5]:
BASE_DIR = "/content/gdrive/MyDrive/Colab Notebooks"
train_csv = os.path.join(BASE_DIR, "train.csv")
test_csv  = os.path.join(BASE_DIR, "test.csv")

df_train = pd.read_csv(train_csv)
df_test  = pd.read_csv(test_csv)

I have saved all of the datasets under ../data/ though I am not sure how this is replicated on google colab - need to check

In [6]:
df_train.head()
df_test.head()

,sample_id,image_path,target_name
0,ID1001187975__Dry_Clover_g,test/ID1001187975.jpg,Dry_Clover_g
1,ID1001187975__Dry_Dead_g,test/ID1001187975.jpg,Dry_Dead_g
2,ID1001187975__Dry_Green_g,test/ID1001187975.jpg,Dry_Green_g
3,ID1001187975__Dry_Total_g,test/ID1001187975.jpg,Dry_Total_g
4,ID1001187975__GDM_g,test/ID1001187975.jpg,GDM_g


I loaded in the test dataset but I did not run my code on it fyi, incase you were wondering

**DATA MANIPULATION**

In [7]:
df_wide = df_train.copy()
df_wide = df_wide.pivot(index = 'image_path', columns = 'target_name', values = 'target').reset_index()
df_wide["cleaned_path"] = df_wide["image_path"].str.replace(r'^train[\\/]', '', regex=True)
df_wide.head()

target_name,image_path,Dry_Clover_g,Dry_Dead_g,Dry_Green_g,Dry_Total_g,GDM_g,cleaned_path
0,train/ID1011485656.jpg,0.0000,31.9984,16.2751,48.2735,16.2750,ID1011485656.jpg
1,train/ID1012260530.jpg,0.0000,0.0000,7.6000,7.6000,7.6000,ID1012260530.jpg
2,train/ID1025234388.jpg,6.0500,0.0000,0.0000,6.0500,6.0500,ID1025234388.jpg
3,train/ID1028611175.jpg,0.0000,30.9703,24.2376,55.2079,24.2376,ID1028611175.jpg
4,train/ID1035947949.jpg,0.4343,23.2239,10.5261,34.1844,10.9605,ID1035947949.jpg


**SET UP FOR CNN / FUNCTIONS FOR CNN**

In [8]:
IMAGE_DIR = f"{BASE_DIR}/train"
assert os.path.exists(IMAGE_DIR), "Image directory NOT found"

df_wide["full_path"] = df_wide["cleaned_path"].apply(
    lambda fname: os.path.join(IMAGE_DIR, fname)
)

df_wide.head(7)

target_name,image_path,Dry_Clover_g,Dry_Dead_g,Dry_Green_g,Dry_Total_g,GDM_g,cleaned_path,full_path
0,train/ID1011485656.jpg,0.0000,31.9984,16.2751,48.2735,16.2750,ID1011485656.jpg,/content/gdrive/MyDrive/Colab Notebooks/train/...
1,train/ID1012260530.jpg,0.0000,0.0000,7.6000,7.6000,7.6000,ID1012260530.jpg,/content/gdrive/MyDrive/Colab Notebooks/train/...
2,train/ID1025234388.jpg,6.0500,0.0000,0.0000,6.0500,6.0500,ID1025234388.jpg,/content/gdrive/MyDrive/Colab Notebooks/train/...
3,train/ID1028611175.jpg,0.0000,30.9703,24.2376,55.2079,24.2376,ID1028611175.jpg,/content/gdrive/MyDrive/Colab Notebooks/train/...
4,train/ID1035947949.jpg,0.4343,23.2239,10.5261,34.1844,10.9605,ID1035947949.jpg,/content/gdrive/MyDrive/Colab Notebooks/train/...
5,train/ID1036339023.jpg,23.0755,2.6135,32.1910,57.8800,55.2665,ID1036339023.jpg,/content/gdrive/MyDrive/Colab Notebooks/train/...
6,train/ID1049634115.jpg,1.5083,3.0167,13.5750,18.1000,15.0833,ID1049634115.jpg,/content/gdrive/MyDrive/Colab Notebooks/train/...


This merges the picture directory to the excel table

In [9]:
TEST_DIR = f"{BASE_DIR}/test"
assert os.path.exists(TEST_DIR), f"Image folder {TEST_DIR} does not exist."


Random check

In [10]:
test_path = TEST_DIR
print("Path:", TEST_DIR)
print("Exists?", os.path.exists(TEST_DIR))

Path: /content/gdrive/MyDrive/Colab Notebooks/test
Exists? True


Tranformation set up for the pictures; to be pre processed before feeding into CNN

Function to Transform the excel table data and picture into meaningful Tensor. Code logic is defined in workflow utils.py

In [12]:
import torchvision.transforms as transforms

IMG_SIZE = 224  # or whatever you already use

train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])

val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])

In [13]:
device = get_device()
kf = create_kfold(n_splits=5)
EPOCHS = 10

for fold, (train_idx, val_idx) in enumerate(kf.split(df_wide)):
    print(f"\n===== Fold {fold + 1} =====")

    train_df = df_wide.iloc[train_idx]
    val_df   = df_wide.iloc[val_idx]

    train_ds = ImageDataset(
        train_df,
        transform=train_tfms
    )
    val_ds = ImageDataset(
        val_df,
        transform=val_tfms
    )

    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
    val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False)

    model = build_model(
        architecture="resnet18",
        num_targets=len(TARGET_COLUMNS),
        weights="IMAGENET1K_V1",
    ).to(device)

    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    train_regression(
    model,
    EPOCHS,
    train_loader,
    val_loader,
    loss_fn,
    optimizer,
    device)



===== Fold 1 =====
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 218MB/s]


Epoch  1/10 | Train Loss: 1155.367995 | Val Loss: 494.733058
Epoch  2/10 | Train Loss: 898.711517 | Val Loss: 555.980876
Epoch  3/10 | Train Loss: 682.402164 | Val Loss: 319.386480
Epoch  4/10 | Train Loss: 526.537899 | Val Loss: 2693.774719
Epoch  5/10 | Train Loss: 367.810382 | Val Loss: 330.251531
Epoch  6/10 | Train Loss: 266.647159 | Val Loss: 345.144872
Epoch  7/10 | Train Loss: 254.579546 | Val Loss: 232.920820
Epoch  8/10 | Train Loss: 233.549652 | Val Loss: 343.722916
Epoch  9/10 | Train Loss: 232.769116 | Val Loss: 689.053670
Epoch 10/10 | Train Loss: 152.702384 | Val Loss: 375.179454

===== Fold 2 =====
Epoch  1/10 | Train Loss: 1069.419630 | Val Loss: 1361.762858
Epoch  2/10 | Train Loss: 807.097470 | Val Loss: 584.145009
Epoch  3/10 | Train Loss: 605.226101 | Val Loss: 542.829417
Epoch  4/10 | Train Loss: 454.834030 | Val Loss: 719.282328
Epoch  5/10 | Train Loss: 318.713564 | Val Loss: 656.172760
Epoch  6/10 | Train Loss: 239.350984 | Val Loss: 625.995168
Epoch  7/10 | Tr

In [ ]:
torch.save(model.state_dict(), "final_model.pth")
print("Model saved to final_model.pth")

Saving the model parameters for easy replication in the future

Without K-fold ensemble.
---


In [14]:
device = get_device()

# Full dataset
full_train_ds = ImageDataset(
    df_wide,
    transform=train_tfms
)

full_train_loader = DataLoader(
    full_train_ds,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

# Build final model
final_model = build_model(
    architecture="resnet18",
    num_targets=len(TARGET_COLUMNS),
    weights="IMAGENET1K_V1"
).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(final_model.parameters(), lr=1e-3)

EPOCHS = 15  # usually a bit higher than CV

print("Training FINAL model on full dataset...")

train_regression(
    final_model,
    num_epochs=EPOCHS,
    train_dl=full_train_loader,
    valid_dl=full_train_loader,  # dummy (not used meaningfully)
    loss_fn=criterion,
    optimizer=optimizer,
    device=device
)

# Save final model
torch.save(final_model.state_dict(), "final_model_full.pth")
print("Saved final model -> final_model_full.pth")


Training FINAL model on full dataset...
Epoch  1/15 | Train Loss: 1153.249868 | Val Loss: 1367.494563
Epoch  2/15 | Train Loss: 867.739972 | Val Loss: 543.093033
Epoch  3/15 | Train Loss: 537.675634 | Val Loss: 315.215720
Epoch  4/15 | Train Loss: 401.599108 | Val Loss: 491.157964
Epoch  5/15 | Train Loss: 281.622896 | Val Loss: 396.997503
Epoch  6/15 | Train Loss: 249.492769 | Val Loss: 313.955555
Epoch  7/15 | Train Loss: 234.476986 | Val Loss: 185.347628
Epoch  8/15 | Train Loss: 150.758182 | Val Loss: 137.124740
Epoch  9/15 | Train Loss: 132.455738 | Val Loss: 161.309380
Epoch 10/15 | Train Loss: 133.206687 | Val Loss: 153.676730
Epoch 11/15 | Train Loss: 104.179278 | Val Loss: 88.924998
Epoch 12/15 | Train Loss: 88.961730 | Val Loss: 76.289083
Epoch 13/15 | Train Loss: 108.346638 | Val Loss: 91.034108
Epoch 14/15 | Train Loss: 86.128791 | Val Loss: 156.534973
Epoch 15/15 | Train Loss: 77.827913 | Val Loss: 132.095476
Saved final model -> final_model_full.pth


In [15]:
device = get_device()

final_model = build_model(
    architecture="resnet18",
    num_targets=len(TARGET_COLUMNS),
    weights=None   # IMPORTANT: None when loading trained weights
).to(device)

final_model.load_state_dict(
    torch.load("final_model_full.pth", map_location=device)
)

final_model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [16]:
eval_ds = ImageDataset(
    df_wide,
    transform=val_tfms
)

eval_loader = DataLoader(
    eval_ds,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

In [17]:
all_preds = []
all_targets = []

with torch.no_grad():
    for images, targets in eval_loader:
        images = images.to(device)
        targets = targets.to(device)

        outputs = final_model(images)

        all_preds.append(outputs.cpu())
        all_targets.append(targets.cpu())

all_preds = torch.cat(all_preds, dim=0)
all_targets = torch.cat(all_targets, dim=0)

In [18]:
r2 = calculate_global_weighted_r2(all_preds, all_targets)
print("Final weighted R²:", r2)

Final weighted R²: 0.7806238506627758


With K-fold ensemble.
---

In [19]:
device = get_device()
kf = create_kfold(n_splits=5)
EPOCHS = 10

fold_models = []

for fold, (train_idx, val_idx) in enumerate(kf.split(df_wide)):
    print(f"\n===== Fold {fold + 1} =====")

    train_df = df_wide.iloc[train_idx]
    val_df   = df_wide.iloc[val_idx]

    train_ds = ImageDataset(train_df, transform=train_tfms)
    val_ds   = ImageDataset(val_df, transform=val_tfms)

    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
    val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False)

    model = build_model(
        architecture="resnet18",
        num_targets=len(TARGET_COLUMNS),
        weights="IMAGENET1K_V1"
    ).to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    train_regression(
        model,
        EPOCHS,
        train_loader,
        val_loader,
        criterion,
        optimizer,
        device
    )

    path = f"model_fold_{fold}.pth"
    torch.save(model.state_dict(), path)
    fold_models.append(path)

    print(f"Saved {path}")


===== Fold 1 =====
Epoch  1/10 | Train Loss: 1159.634488 | Val Loss: 922.002035
Epoch  2/10 | Train Loss: 865.372281 | Val Loss: 416.601547
Epoch  3/10 | Train Loss: 639.176731 | Val Loss: 968.412811
Epoch  4/10 | Train Loss: 466.535099 | Val Loss: 632.099620
Epoch  5/10 | Train Loss: 340.558741 | Val Loss: 429.351847
Epoch  6/10 | Train Loss: 264.719347 | Val Loss: 330.271718
Epoch  7/10 | Train Loss: 204.508552 | Val Loss: 308.602676
Epoch  8/10 | Train Loss: 162.982101 | Val Loss: 2765.247843
Epoch  9/10 | Train Loss: 147.747585 | Val Loss: 178.490901
Epoch 10/10 | Train Loss: 110.270524 | Val Loss: 269.603182
Saved model_fold_0.pth

===== Fold 2 =====
Epoch  1/10 | Train Loss: 1076.108588 | Val Loss: 918.714274
Epoch  2/10 | Train Loss: 811.878052 | Val Loss: 677.087260
Epoch  3/10 | Train Loss: 601.504164 | Val Loss: 515.045441
Epoch  4/10 | Train Loss: 433.828871 | Val Loss: 1150.973775
Epoch  5/10 | Train Loss: 303.864266 | Val Loss: 796.831116
Epoch  6/10 | Train Loss: 219.900

In [21]:
# ---- K-fold ensemble R² evaluation ----

device = get_device()

# 1. Evaluation dataset (same samples for all models)
eval_ds = ImageDataset(
    df_wide,
    transform=val_tfms
)

eval_loader = DataLoader(
    eval_ds,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

ensemble_preds = []

# 2. Load each fold model and predict
for fold, path in enumerate(fold_models):
    print(f"Loading {path}")

    model = build_model(
        architecture="resnet18",
        num_targets=len(TARGET_COLUMNS),
        weights=None  # IMPORTANT when loading trained weights
    ).to(device)

    model.load_state_dict(torch.load(path, map_location=device))
    model.eval()

    preds = []

    with torch.no_grad():
        for images, _ in eval_loader:
            images = images.to(device)
            outputs = model(images)
            preds.append(outputs.cpu())

    preds = torch.cat(preds, dim=0)
    ensemble_preds.append(preds)

# 3. Average predictions across folds
ensemble_preds = torch.stack(ensemble_preds, dim=0)   # (n_folds, N, targets)
final_preds = ensemble_preds.mean(dim=0)              # (N, targets)

# 4. Get true targets
all_targets = torch.tensor(
    df_wide[TARGET_COLUMNS].values,
    dtype=torch.float32
)

# 5. Compute weighted R²
r2 = calculate_global_weighted_r2(final_preds, all_targets)

print("K-fold ensemble weighted R²:", r2)

Loading model_fold_0.pth
Loading model_fold_1.pth
Loading model_fold_2.pth
Loading model_fold_3.pth
Loading model_fold_4.pth
K-fold ensemble weighted R²: 0.8712823061179886
